In [ ]:
# !pip install tensorflow_addons
# !pip install transformers
# !pip install kss
# !pip install emoji==1.2.0

In [166]:
import os
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import tensorflow_addons as tfa
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification,TFBertModel
import warnings
warnings.filterwarnings(action='ignore') 

### Load Tokenizer from KlueBert-base Model

In [167]:
model_name = "klue/bert-base"
tokenizer = BertTokenizer.from_pretrained(model_name)

### Reviews Proprecessing

In [6]:
# 학습하지 않은 리뷰로 분류 결과 보기

test_reviews_raw = pd.read_csv('./raw_test_reviews.csv')
test_reviews_raw  #list_review = 해당 리뷰가 달린 제품의 class

,Unnamed: 0,review,list_review
0,0,💜구매동기💜,0
1,1,웰라쥬는 원래 수분라인 제품으로 유명하다고 알고 있어서 진정제품은 잘 알지 못했는데...,0
2,2,💜사용후기💜,0
3,3,일단 사용후기는 정말 만족도 최상입니다!!!,0
4,4,제가 원래 그날 기간에는 얼굴이 완전 뒤집어지고 피부 컨디션도 안좋아지는데 이 크림...,0
...,...,...,...
5942,7174,처음에 썼을 때는 이마에 있는 여드름이 많이 가라앉았었는데 일시적일 뿐 다시 올라오...,29
5943,7175,볼쪽에도 쓰기전에는 많이 없었는데 이거 바르고 나니깐,29
5944,7176,너무 올라와서 원래 쓰던 거만 쓰니까 바로 가라앉더라고요ㅋㅋ,29
5945,7177,제 피부한테 안맞는 거 겠지만 후기올려봅니다,29


In [7]:
# 리뷰와 class를 tuple로 묶어 처리하기

split_reviews = []

for i in range(30):
    txt = test_reviews_raw.loc[test_reviews_raw.list_review == i].review
    tup = (list(txt), i)
    split_reviews.append(tup)
    
len(split_reviews)

30

In [347]:
from reviews_preprocessing import reviews_prepro # 전처리 모듈 import

In [12]:
reviews = []
for i in range(30):
    temp = reviews_prepro(split_reviews[i][0])
    reviews.append((temp, split_reviews[i][1]))

[Kss]: Because there's no supported C++ morpheme analyzer, Kss will take pecab as a backend. :D
For your information, Kss also supports mecab backend.
We recommend you to install mecab or konlpy.tag.Mecab for faster execution of Kss.
Please refer to following web sites for details:
- mecab: https://cleancode-ws.tistory.com/97
- konlpy.tag.Mecab: https://uwgdqo.tistory.com/363



In [13]:
reviews[0][0]

['같은 게 진짜 진짜 진~짜 좋아요',
 '건성피부인데 흡수력이 좋으면서',
 '별점을 준건 민감성인 제가 쓰기에도 순해서 입니다',
 '보습력도 쏘쏘했고 끈쩍거리지 않아서 좋았는데 성분중에 안 맞는 게 있었나봐요',
 '️진정효과 대단할까',
 '뾰루지 짜고 나서 패치 붙이기 전에 진정시키고 싶어서 산 거라 꾸준히 발라봤지만 정말 효과가 없었어요',
 '저는 지복합성인데 참 좋더라구요',
 '겨울에 쓰긴 좀 가벼워서 여름되면 다시 쓰려고요',
 '좀 건조해요',
 '시카크림이라고 하는데시카는 진정 효과에 좋은건데 진정은 따로 되지 않았습니다',
 '피부가 일어났을때 좋다고 햐서 사봤어요',
 '제품 발림성이라던지 그런건 다 좋았어요',
 '지금까지 몇통째 쓰고있는ㅈㅣ',
 '하루 아침에 막 달라지는 드라마틱한 진정효과는 아니고 붉은기가 살짝 개선 되는 정도의 진정효과를 가지고 있어용',
 '큰 기대를 가지고 쓰기보다는 데일리로 가볍게 사용할 진정 크림으로 사용하는 게 맞는 거 같아용',
 '수분이 채워져서 좋아요',
 '자극없이 순한 건 맞는데',
 '모공을 막는 느낌이에요',
 '너무너무 자극없이 순해서 아주 잘쓰고있는즁',
 '말이 필요없죠~',
 '만족 했는데 다바르고나니 유분이 정말 많이 느껴졌어요',
 '진정도 좋다고는 못느꼈습니다',
 '뻑뻑하지 않고 스르륵 막 발라도 잘발리고 보습도 어느정도 되기 때문인데요',
 '발림성도 좋아요',
 '시카제품 찾다가 후기가 좋아서 구매해봤어요',
 '간단한 팩으로 사용하기 좋아 잘 썼어요',
 '재구매 각입니다',
 '피부에 유분기 남기지 않고 흡수돼서 화장 전에 발라도',
 '제가 원래 그날 기간에는 얼굴이 완전 뒤집어지고 피부 컨디션도 안좋아지는데 이 크림 바르고 부터는 완죤 피부에 뭐가 잘 안나요',
 '이제 한 2주정도 사용한거 같은데 트러블이 진정되길 바라며',
 '피부가 예민한 편이라 아무거나 못 쓰는 편이라',
 '한 번쯤 사볼만 하긴 한데 재구매는 안할 것 같은 제품',
 '생각보다 괜찮고 잘

### Load Models

In [169]:
from tensorflow.keras.models import load_model

key_sent_model = load_model('./final_key_sent_model.h5', compile=False,
                    custom_objects={'TFBertForSequenceClassification': TFBertForSequenceClassification})

sentiment_model = load_model('./final_sentiment_model.h5', compile=False,
                     custom_objects={'TFBertForSequenceClassification': TFBertForSequenceClassification})

### Convert to Model Input Format

In [170]:
def convert_data(X_data):
    MAX_SEQ_LEN =  260
    tokens, masks, segments = [], [], []
    
    for X in tqdm(X_data):   
        token = tokenizer.encode(X, truncation = True, padding = 'max_length', max_length = MAX_SEQ_LEN)
        num_zeros = token.count(0)
        mask = [1] * (MAX_SEQ_LEN - num_zeros) + [0] * num_zeros    
        segment = [0]*MAX_SEQ_LEN
        tokens.append(token)
        masks.append(mask)
        segments.append(segment)

    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)

    return [tokens, masks, segments]

### Prediction

In [171]:
def get_sentiment(review_list):
    
    # key sentences classification
    x1 = review_list
    x1_ = convert_data(x1)
    predicted_value1 = key_sent_model.predict(x1_)
    predicted_label1 = np.argmax(predicted_value1, axis = 1)
    
    predicted_review=[]
    for i in range(len(x1)):
        if predicted_label1[i] == 1:
            predicted_review.append(np.array(x1)[i])
    
    # positive / negative classification
    x2 = convert_data(predicted_review)
    predicted_value2 = sentiment_model.predict(x2)
    predicted_label2 = np.argmax(predicted_value2, axis = 1)
    
    pred_p = [] # positice
    pred_n = [] # negative
    for i in range(len(predicted_review)):
        if predicted_label2[i] == 1:
            pred_p.append(np.array(predicted_review)[i])
        else:
            pred_n.append(np.array(predicted_review)[i])
            
    return pred_p, pred_n

In [25]:
# predict 결과 dictionary에 저장 - key: class, value: (positive, nagative)
reviews_dic = {}

for i in range(30):
    positive, negative = get_sentiment(reviews[i][0])
    reviews_dic[i] = (positive, negative)

100%|██████████████████████████████████████████████████████████████████████████████| 171/171 [00:00<00:00, 2375.00it/s]


In [29]:
reviews_dic[5][0]  # class 5번의 긍정 리뷰

['화장솜 같이 들어 있어서 혜자구성이 네요',
 '크림의 향도 너무 거북하지 않고 너무 좋았어요',
 '보습 및 탄력 부분에서는 진심 촉촉 고보습함이 느껴져서 이런 거 좋아하시는 분이라면 추천해드려요',
 '모공이 더 넓어지는 듯하게 보이더라구요',
 '️우선 저는 건성~악건성의 피부타입인데 그런 저에게 아주 만족할만한 보습력이엇어요',
 '가을 겨울이 더 잘 맞을 것 같긴 하네요',
 '추천대상은 40대 이상 처지는 피부와 주름이 고민되시는 분들이에요 부모님 선물로도 매우 좋을 것 같습니다',
 '향은 비슷한 것 같고 제형도 비슷해요',
 '더 쫀쫀해진만큼 보습도 좀 더 오래 유지되는 것 같아요~',
 '뭔가 찐득한 느낌이 아니라 가지고있는 유분감에 비해 보송하다고 느껴져요',
 '끈적임 산뜻함 마무리감은 또 너무 끈적이지는 않았습니다',
 '엄마는 향이 좋다고 넘 좋아하시더라구요',
 '리뉴얼 전 제품보다 확실히 무겁고 발림성도 더 리치해진 느낌?',
 '기존 크림은 유분이 많아도 흡수력이 좋아서 번들거리는 느낌보다는 피부가 좋아졌다 라는 느낌이 강했습니다',
 '적당히 크리미하게 꾸덕한 정도도 비슷하고 발림성도 거의 똑같고요',
 '각질이 일어나 번들거려도 마법같이 가라앉혀주는 크림이었고 이것만 쓰다가 다른것좀 써보자며 바이오힐보 로 갈아타게 되었습니다',
 '주름 개선이나 리프팅 효과가 더 있을 것 같은 느낌의 제형이에요',
 '바르고 나면 피부가 쫀쫀하고 탄력적이게 바뀌더라구요',
 '50대이상 여성분들께 좀 건조하다싶은분께 추천드릴게요',
 '무겁지 않은 영양크림으로 발림성도 챱챱 잘 발라짐',
 '악건성 피부에는 얇게 2번 나눠서 발라줬더니 아침까지 촉촉한 느낌이 느껴졌어요',
 '️흡수까지는 좀 걸리고 보습감이 진짜 좋아요',
 '지성 복합성에게 더 추천',
 '보통 리뷰에 향기가 별로고 너무 강하다는 평이 많아서 걱정했는데 생각보다 괜찮았어요',
 '엄마는 건성 저는 복합성인데 제가 사용하기에도 얇게만 발라주면 그렇게 많이 기름지지는 않아서 

In [30]:
reviews_dic[5][1]  # class 5번의 부정 리뷰

['혹시나해서 사봤는데 역시나 화장품으로 미백 주름개선을 하는 것에는 한계가 있다는 걸 말씀드리고 후기 시작하겠습니다',
 '이건 사계절 발라도 괜찮긴 한데 지성이신 분들은 여름엔 조금 답답함감?',
 '홍조에는 도움이 되지 않는듯합니다',
 '주름 개선 효과는 아직 모르겠어요',
 '탄력 개선 면에서는 크게 효과를 느끼지는 못했다고 합니다',
 '모공관리에 유명하다고 해서 사봤는데 늘어나지 않는게 느껴졌어요',
 '피부가 좀 민감하고 특히 유분기 많은 화장품이랑 안맞는 분들에게는 비추입니다',
 '악건성인 편인데 잘못 샀나보네요',
 '이 제품은 흡수력이 떨어지고 유분이 많다보니 화장을 했을 때 밀립니다',
 '이 제품은 굉장히 되직한 제형이라 지성/여드름/모공넓은 분들에게는 딱히 추천드리고 싶지 않고 그래도 사고 싶으시다면 모공이 넓지 않은 눈밑 주름 목 주름 쪽에만 바르는 것 추천드려요',
 '약간 겨울철 바디로션 향기인데 그게 너무 강해서 향에 민감하신 분들은 머리가 아프실 수도 있습니다',
 '유분이 많은 것에 비해서 각질을 잠재우는 건 부족했어요',
 '저는향이 너무 강하다고 느꼇습니다 약간 엄마 화장품 냄새엿어요',
 '흡수력도 전 제품이 좀 더 빠르게 흡수가 잘 되었던 것 같아 아쉬워요',
 '지성피부이신 분들은 추천하지 않고 수부지 중복합 피부도 계절감 바르는 양 같이 바르는 제품 등을 고려해서 바르시는 게 좋을 거 같아요',
 '바이오 힐보는 기존 크림에 비해 유분감이 더 많은편인거같으며 흡수력이 다소 떨어집니다',
 '제가 사용했을 때는 바로 다음날 트러블들이 짜잘짜잘하게 올라왔어요',
 '지복합성인 저에게는 겨울에 좋을꺼같으나 여름에는 다소 과한듯합니다',
 '너무 인위적인 향이라서 계속 맡으면 멀미할 정도의 향이었어요',
 '왜인지 따가울 때가 있어요',
 '좋다고해서써보긴하는데 건성인 저에게는 보습 탄력이되는지는 모르겠어요',
 '많이 펴발라도 그게 잘 없어지지 않아서 아쉬웠어요',
 '진짜 저는 향이 너무 불호여서 3점 준거예요',


In [28]:
# json file 로 결과 저장

import json
with open('review_result.json', 'w') as fp:
    json.dump(reviews_dic, fp)

### # text file predict

In [417]:
# 리뷰 불러오기

with open('./sample.txt') as f:
    reviews = [line.rstrip() for line in f]

print(f"reviews size : {len(reviews)}")

reviews size : 5


In [418]:
review_list = reviews_prepro(reviews)  # 리뷰 전처리
good, bad = get_sentiment(review_list)  # 분류 모델 2가지 통과 : 핵심문장 추출 -> 긍/부정

100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 797.78it/s]


In [419]:
good

['연고같은 제형이라 바르고 흡수 잘 시켜야하고 너무 많이 바르면 허옇게 얼굴 변신이 짠 한번에 많은 양을 바르기보다 쌓는 형식으로 한번 바르고 흡수 시켜주면서 건조한 부분만 따로 덧 발라 주세요',
 '트러블 올라온거 다른 제품 사용했을때보다 효과가 좀 빨랐어요',
 '많이 바른다고 좋은게 아니기도 하고 크림 제형이 일반 크림보다 좀 더 꾸덕한??']

In [420]:
bad

['오전에는 화장 때문에 너무 많이 바르면 밀리기도 하고 답답해서 오전에는 트릭세라 로션 두번 바르고 건조한 부분에만 크림 얇게 발라요']